## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_30_copy.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_30_copy.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "단일_배아_이식_여부",
    "착상_전_유전_진단_사용_여부",
    "배아_생성_주요_이유",
    "총_생성_배아_수",
    "미세주입된_난자_수",
    "미세주입에서_생성된_배아_수",
    "이식된_배아_수",
    "미세주입_배아_이식_수",
    "저장된_배아_수",
    "미세주입_후_저장된_배아_수",
    "해동된_배아_수",
    "해동_난자_수",
    "수집된_신선_난자_수",
    "저장된_신선_난자_수",
    "혼합된_난자_수",
    "파트너_정자와_혼합된_난자_수",
    "기증자_정자와_혼합된_난자_수",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "동결_배아_사용_여부",
    "신선_배아_사용_여부",
    "기증_배아_사용_여부",
    "대리모_여부",
    "ICSI_배아_이식_비율",
    "ICSI_배아_생성_비율",
    "ICSI_성공률"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

I 2025-02-14 21:28:31,186] Trial 155 finished with value: 0.7379094441079711 and parameters: {'n_estimators': 7534, 'num_leaves': 51, 'learning_rate': 0.0006466079722720145, 'reg_alpha': 5.390917786114456e-06, 'reg_lambda': 0.00014388089013544448}. Best is trial 155 with value: 0.7379094441079711.

In [7]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 2000, 10000),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 10.0, log=True),

        'random_state': 42,
        # 'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1200)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-14 12:33:57,666] A new study created in memory with name: no-name-0cc245da-f458-44a6-b399-996f6a33fae9
[I 2025-02-14 12:42:35,409] Trial 0 finished with value: 0.7264298444845337 and parameters: {'n_estimators': 8872, 'num_leaves': 411, 'learning_rate': 2.2984588509821827e-06, 'reg_alpha': 0.00014946697240810262, 'reg_lambda': 1.2466761684573362}. Best is trial 0 with value: 0.7264298444845337.
[I 2025-02-14 12:48:34,671] Trial 1 finished with value: 0.7335722868953221 and parameters: {'n_estimators': 6829, 'num_leaves': 349, 'learning_rate': 0.00095017845187706, 'reg_alpha': 0.17679049775668296, 'reg_lambda': 3.9848293927775545e-06}. Best is trial 1 with value: 0.7335722868953221.
[I 2025-02-14 12:54:30,470] Trial 2 finished with value: 0.7289194376006602 and parameters: {'n_estimators': 8664, 'num_leaves': 215, 'learning_rate': 1.5099953130336956e-06, 'reg_alpha': 3.9607381323220947e-05, 'reg_lambda': 5.079199618636747e-05}. Best is trial 1 with value: 0.7335722868953221.


KeyboardInterrupt: 

.